<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/master/Fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the Perpetrator's gender using an evolutionary algorithm to select features.

## - Fitness

This code has all steps for Fitness. Where:

* Imports: the necessary libraries and the population; 
* Scripts: steps for do mutation or crossover;
* Setups: results.

In [ ]:
!pip install category_encoders

## Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import read_csv 
from category_encoders import TargetEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt

## Scripts

In [3]:
def database(url):
  df_encoded = pd.read_csv(url)
  df_encoded = df_encoded.drop(columns=['Unnamed: 0'])

  return df_encoded

In [4]:
def fitness(population,df,models):
  result = []

  x = df[df.columns.difference(['Victim Sex'])]
  y = df[['Victim Sex']]
  
  x_train, x_test, y_train, y_test = np.array(train_test_split(x, y, test_size=0.2, stratify=y, random_state=42))

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.ravel(y_train)
  y_test = np.array(y_test)

  for p in population:
    model = models[p[1]]
    ind = [i==1 for i in p[1:]] 
    model.fit(x_train[:,ind],y_train)
    y_pred = model.predict(x_test[:,ind])
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    fit_value = 20*rmse + np.sum(p[1:])

    result.append([fit_value,p])
  return result

## Setups

In [5]:
with open('/content/drive/My Drive/Data Files/initial_population.txt', "rb") as fp:   
  population = pickle.load(fp)

In [6]:
models = [DecisionTreeClassifier(max_depth=14),
          DecisionTreeClassifier(max_depth=10),
          DecisionTreeClassifier(max_depth=3),
          RandomForestClassifier(max_depth=14),
          RandomForestClassifier(max_depth=10)]

In [7]:
df_encoded = database('/content/drive/My Drive/Data Files/df_model_per_encoded')

In [8]:
model_fitness = fitness(population,df_encoded,models)

In [12]:
pd.DataFrame(model_fitness, columns=['Fitness','Individuo'])

,Fitness,Individuo
0,16.423973,"[2, True, True, True, False, False, True, Fals..."
1,18.528651,"[0, True, True, True, False, True, False, True..."
2,19.457983,"[2, True, True, False, True, True, False, True..."
3,14.518241,"[4, False, True, True, True, False, False, Fal..."
4,14.529899,"[5, False, False, True, True, False, True, Tru..."
...,...,...
95,20.516920,"[4, True, True, True, True, True, True, True, ..."
96,18.518658,"[5, False, True, True, True, False, True, Fals..."
97,16.424638,"[0, False, False, True, False, False, True, Fa..."
98,17.546106,"[5, False, False, False, False, True, True, Fa..."


In [23]:
#export 
with open("/content/drive/My Drive/Data Files/fitness.txt", "wb") as fit: 
  pickle.dump(model_fitness, fit)